<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Veracity57.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import fastai

In [0]:
from fastai.text import * 

In [0]:
def read(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 3.0:
            return 'Query'
        elif col['label'] == 2.0:
            return 'Deny'
        elif col['label'] == 1.0:
            return 'Support'
        elif col['label'] == 0.0:
            return 'Comment'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [0]:
# For veracity
train = read_veracity('train.csv', True)
test = read_veracity('test.csv')

In [0]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [7]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [8]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [11]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.124063,4.512078,0.306920,00:00


In [12]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.773234,3.761905,0.354241,00:00


In [13]:
learn.predict("This is a review about", n_words=10)

'This is a review about the way people being today come to be : including'

In [0]:
learn.save_encoder('ft_enc')

In [15]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (217 items)
x: TextList
xxbos xxmaj xxunk xxup u.s. official : xxmaj canadian xxunk has xxunk xxup u.s. that one shooter is dead in xxmaj ottawa . xxmaj live blog : http : / / t.co / q98amohu7 t,xxbos xxmaj hero . xxunk : xxmaj xxunk xxmaj xxunk , head of security in parliament , being xxunk for taking down shooter # xxmaj ottawa http : / / t.co / xxunk ",xxbos xxmaj five hostages have escaped the xxunk xxmaj lindt xxmaj cafe in # xxmaj sydney | xxmaj our latest update : http : / / t.co / xxunk http : / / t.co / xxunk,xxbos xxmaj several more hostages xxunk from cafe under siege in xxmaj sydney , local media say http : / / t.co / xxunk,xxbos # xxup breaking : xxmaj police have confirmed that the # sydneysiege is over . # 9news
y: CategoryList
True,True,True,Unverified,True
Path: .;

Valid: LabelList (55 items)
x: TextList
xxbos xxup update : xxmaj gunman in kosher grocery store is demanding the release of the suspects in # charlieheb

In [16]:
data_clas.show_batch()

text,target
"xxbos xxup developing : xxmaj gunman xxmaj xxunk xxmaj hostages xxmaj in xxmaj sydney xxmaj cafe , ' xxup isis ' xxmaj flags xxmaj being xxmaj held xxmaj against xxmaj window - http : / / t.co / xxunk http : / / t.co / xxunk",True
xxbos xxmaj xxunk unarmed xxunk . \n xxmaj xxunk xxunk . \n xxmaj xxunk xxunk xxunk . \n xxmaj xxunk xxunk . \n xxmaj xxunk the victim . \n xxmaj xxunk . xxmaj police . xxmaj xxunk . \n # xxmaj ferguson # mikebrown,Unverified
"xxbos xxmaj xxunk like every xxmaj charlie xxmaj hebdo xxunk has been killed : 1 . xxmaj charb - was on xxmaj al - xxmaj xxunk 's xxunk xxunk ; 2 . xxmaj cabu , 3 . xxmaj wolinski , 4 . xxmaj tignous",Unverified
xxbos xxmaj this is the soldier who died today at the # xxmaj ottawa xxmaj war xxmaj memorial . xxmaj his name is xxmaj cpl . xxmaj nathan xxmaj cirillo . # xxup xxunk # xxunk http : / / t.co / xxunk,True
"xxbos xxmaj xxunk xxmaj xxunk , xxmaj hostage - taker in # xxmaj sydney : xxmaj xxunk # xxmaj xxunk xxunk in xxunk - xxunk xxunk xxunk xxunk http : / / t.co / xxunk http : / / t.co / xxunk",True


In [17]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.248131,1.039875,0.527273,00:00


In [18]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,1.057812,1.018918,0.490909,00:00


In [19]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.847361,0.976003,0.490909,00:00
1,0.870863,0.931435,0.545455,00:00
2,0.829450,0.955331,0.545455,00:00
3,0.765814,0.930448,0.545455,00:00
4,0.695729,0.891227,0.581818,00:00
5,0.623314,0.896866,0.636364,00:00
6,0.550689,0.879746,0.672727,00:00
7,0.496654,0.914474,0.672727,00:00
8,0.455816,0.919898,0.690909,00:00
9,0.423889,0.923829,0.654545,00:00


In [0]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [22]:
accuracy_score(test['label'], l)

0.5714285714285714

In [23]:
f1_score(test['label'], l, average='macro')

0.5650623885918004

In [0]:
learn.export('veracity.pkl')